In [1]:
# Important necessary libraries
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
# Seasons of NBA games we will scrape
SEASONS = list(range(2016, 2023))

In [3]:
# Create directories
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [4]:
# Get html for a given URL and selector. Return the html
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    # Sleep at least 5 seconds (more on retries) because we can get banned from scraping the site too quickly.
    for i in range(1, retries+1):
        time.sleep(sleep * i)
        
        # Get html using playwright which operates asynchronously. This is why we use async and await
        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch() # Used Firefox since Chromium wasn't working
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title()) # Print page title to let us know which page we've got
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout error on {url}")
            continue
        else:
            break
    return html

In [5]:
# For an inputted season, get the relevant data for each month and save it to the standings folder
async def scrape_season(season):
    # Get HTML of inputted season using get_html function
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    # Get links to months in the inputted season using Beautiful Soup
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://www.basketball-reference.com{l}" for l in href]
    
    #For each month, get the .html with the id of all_schedule, and save it to a file in the standings folder
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
# for season in SEASONS:
#     await scrape_season(season)

In [7]:
standings_files = os.listdir(STANDINGS_DIR)

In [8]:
# For a month of NBA games (inputted as standings_file), scrape the boxscore for each game. 
# Save the .html for each game in a file in the scores folder.
async def scrape_game(standings_file):   
    with open(standings_file, 'r') as f:
        html = f.read()

    # Use BeautifulSoup to find all the boxscore .html links in the inputted file. Create a list with all found boxscores.
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    # For each boxscore, get the .html with the id of content and write it to the scores folder. 
    # If the boxscore already exists, move onto the next one.
    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [9]:
# Filter out any non-HTML files (.DS_Store)
standings_files = [s for s in standings_files if ".html" in s]

In [13]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)

    await scrape_game(filepath)